# Motivation
I'm thinking of creating a simulation of players games.
- Minutes
- Poisson FGA|minutes
- Binomial FGM|minutes

The first step is to get an idea of how many minutes a player plays in a game

In [5]:
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
# Import class
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
class_dir = os.path.join(parent_dir, 'Scripts')
sys.path.insert(0, class_dir)
import NBAStats
import numpy as np
import datetime as dt
import pandas as pd
stats_file = os.path.join(parent_dir, 'Data', 'nba_stats.pickle')
print(stats_file)
nba_stats = pickle.load(open(stats_file, 'rb'))

C:\Users\User\Desktop\nba_stats\Data\nba_stats.pickle


In [6]:
def get_player_df(nba_stats, player, player_id=False):
    ''' Get the dataframe containing info on referenced player. If player_id=True, player id given instead of name'''
    if player_id:
        if player not in nba_stats.get_player_stats().keys():
            raise ValueError('player_id {} not found'.format(player))
        
        return nba_stats.get_player_stats()[player].sort_index(ascending=False)
    else:
        if player not in nba_stats.get_player_name_id_dict().keys():
            raise ValueError('player name {} not found'.format(player))
        
        return nba_stats.get_player_stats()[nba_stats.get_player_name_id_dict()[player]].sort_index(ascending=False)

In [7]:
def player_df_convert_numeric(player_df):
    ''' Returns a dataframe that converts relevant columns to numeric 
    Columns converted:
    MIN
    '''
    player_df['MIN'] = player_df['MIN'].apply(
        lambda str_min: int(str_min[:-3]) + int(str_min[-2:])/60.0 if str_min is not None else 0 )

    return player_df

In [8]:
df_games = nba_stats.get_games()
df_games.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,HOME_AWAY,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID,OPP_PTS
0,2018-19,1610612738,BOS,0021800001,2018-10-16T00:00:00,BOS vs. PHI,W,105,HOME,PHI,1610612755,87
1,2018-19,1610612744,GSW,0021800002,2018-10-16T00:00:00,GSW vs. OKC,W,108,HOME,OKC,1610612760,100
2,2018-19,1610612760,OKC,0021800002,2018-10-16T00:00:00,OKC @ GSW,L,100,AWAY,GSW,1610612744,108
3,2018-19,1610612755,PHI,0021800001,2018-10-16T00:00:00,PHI @ BOS,L,87,AWAY,BOS,1610612738,105
0,2018-19,1610612763,MEM,0021800005,2018-10-17T00:00:00,MEM @ IND,L,83,AWAY,IND,1610612754,111
